In [ ]:
# Parameters cell (required by Papermill)
organism = None
sample_id = None
source_image_path = None
square_002_bin_path = None
min_cells = None
min_genes = None
top_variable_genes = None
microns_per_pixel = None
working_dir = None
metadata = None

# Cell Binning and QC Workflow
This notebook performs spatial transcriptomics cell segmentation and quality control using the `bin2cell` pipeline, following a structure similar to prior internal notebooks. Below, we load raw Visium HD data, perform segmentation using StarDist, convert bin-level data to cell-level data, apply quality control and normalization, and visualize the results.

## 1. Load and Inspect Raw Data

In [ ]:
from pathlib import Path
from typing import Dict, Tuple, Optional
from pprint import pprint
from abc import ABC, abstractmethod
from dataclasses import dataclass
from PIL import Image

import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
import scanpy as sc
import squidpy as sq
import numpy as np
import pandas as pd

import bin2cell as b2c

In [ ]:
# load 002um
adata = b2c.read_visium(
    path=square_002_bin_path,
    source_image_path=source_image_path,
    genome="mm10"
)
adata.var_names_make_unique()
adata

In [ ]:
# convert to int32
adata.X = adata.X.astype(np.int32)

In [ ]:
#histogram og
adata.obs['n_genes'] = (adata.X > 0).sum(axis=1).A1 if hasattr(adata.X, "A1") else (adata.X > 0).sum(axis=1)
plt.hist(adata.obs['n_genes'], bins=100)
plt.xlim(0, 500)
plt.xlabel("Number of genes")
plt.ylabel("Number of barcodes")
plt.title("Gene count distribution per barcode")

In [ ]:
# basic filtering
adata.raw = adata.copy()
sc.pp.filter_cells(adata, min_genes=min_genes)
sc.pp.filter_genes(adata, min_cells=min_cells)
adata

In [ ]:
#histogram min_genes==30
plt.hist(adata.obs['n_genes'], bins=100)
plt.xlim(0, 500)
plt.xlabel("Number of genes")
plt.ylabel("Number of barcodes")
plt.title("Gene count distribution per barcode")

## 2. Quality Control Metrics
Calculate standard QC metrics for cell-level filtering and downstream analysis.

In [ ]:
# compute quality control metrics
sc.pp.calculate_qc_metrics(adata, inplace=True)

In [ ]:
plt.hist(adata.obs.loc[adata.obs['in_tissue'] == 1, 'n_genes'], bins=100)
plt.xlim(0, 500)
plt.xlabel("Number of genes (in_tissue=1)")
plt.ylabel("Number of barcodes")
plt.title("Gene count per in-tissue barcode")

In [ ]:
# additional QC Metrics
# total umi counts per barcode
adata.obs['total_counts'].hist(bins=100)
plt.xlim(0, 500)
plt.xlabel("Total UMI counts")
plt.ylabel("Barcodes")
plt.title("UMI distribution per barcode")

In [ ]:
# correlation between umi counts and gene counts
sc.pl.scatter(adata, x='total_counts', y='n_genes')

## 3. Generate Scaled HE Image
This step creates a high-resolution version of the H&E image used for downstream segmentation.

In [ ]:
import os

# Define output directory for StarDist
stardist_dir = "stardist"
os.makedirs(stardist_dir, exist_ok=True)

# Parameters
mpp = float(microns_per_pixel)  # from pipeline input
crop = True                     # always crop to buffer zone for consistency
save_path = os.path.join(stardist_dir, f"{sample_id}_he_crop.tiff")

# Format the image key based on mpp and whether a crop buffer is used
image_key = f"{mpp}_mpp"
if crop:
    image_key += "_150_buffer"

# Save it for reference inside the AnnData object
adata.uns["b2c_image_key"] = image_key

# Generate the scaled HE image
b2c.scaled_he_image(
    adata=adata,
    mpp=mpp,
    crop=crop,
    save_path=save_path
)

In [ ]:
# Retrieve the library ID (usually the first and only one)
library_id = list(adata.uns['spatial'].keys())[0]

# Retrieve the image key from your stored metadata
image_key = adata.uns.get("b2c_image_key", "0.25_mpp_150_buffer")  # fallback just in case

# Retrieve spatial basis that matches the image_key (if known)
spatial_key = None
for key in adata.obsm.keys():
    if image_key in key:
        spatial_key = key
        break

if spatial_key is None:
    spatial_key = "spatial"  # fallback if nothing matches

# Plot dynamically retrieved image and barcode overlay
plt.imshow(adata.uns['spatial'][library_id]['images'][image_key], cmap='gray')
plt.scatter(adata.obsm[spatial_key][:, 0], adata.obsm[spatial_key][:, 1], s=1, c="red")
plt.gca().invert_yaxis()
plt.title("Barcode overlay on scaled HE image")
plt.show()

In [ ]:
# let's destripe our data
b2c.destripe(adata, counts_key='total_counts', adjust_counts=True)

In [ ]:
# Get row and column range from adata
row_min, row_max = adata.obs["array_row"].min(), adata.obs["array_row"].max()
col_min, col_max = adata.obs["array_col"].min(), adata.obs["array_col"].max()

print(f"Row range: {row_min} to {row_max}")
print(f"Col range: {col_min} to {col_max}")

# Calculate center
row_center = (row_min + row_max) // 2
col_center = (col_min + col_max) // 2

# Define half-size of the zoom box (150x150 region)
half_window = 75

# Create a mask for zoomed region
zoom_mask = (
    (adata.obs['array_row'] >= row_center - half_window) &
    (adata.obs['array_row'] <= row_center + half_window) &
    (adata.obs['array_col'] >= col_center - half_window) &
    (adata.obs['array_col'] <= col_center + half_window)
)

# Subset and copy
bdata_zoomed = adata[zoom_mask].copy()

# Visualize the destriped image and normalized signal
sc.pl.spatial(
    bdata_zoomed,
    color=["total_counts", "n_counts_adjusted"],  # Update with any feature of interest
    img_key=image_key,  # use your dynamically stored image_key
    cmap="Reds",
    ncols=2
)

## 4. Run StarDist Segmentation
Apply the trained StarDist model to extract nuclear labels from the H&E image.

In [ ]:
# Derive paths dynamically from image key
he_crop_path = os.path.join("stardist", f"{sample_id}_he_crop.tiff")
stardist_output_path = os.path.join("stardist", f"{sample_id}_he_crop.npz")

# # run stardist
b2c.stardist(
    image_path=he_crop_path,              # your high-res, scaled H&E image
    labels_npz_path=stardist_output_path,          # where to save the segmentation result
    stardist_model="2D_versatile_he",           # pretrained model for H&E
    prob_thresh=0.01                            # low threshold is typical for tissue
)

## 5. Inject Labels into AnnData
Store the predicted StarDist labels into the AnnData object for bin-to-cell transformation.

In [ ]:
# Define paths and keys dynamically
labels_key = "labels_he"  # consistent key for annotations
basis_key = "spatial_cropped_150_buffer"  # or adapt to reflect actual key from scaled image

# Insert StarDist labels into the AnnData object
b2c.insert_labels(
    adata=adata,
    labels_npz_path=stardist_output_path,
    basis="spatial",  # this is the default spatial obsm used in b2c
    spatial_key=basis_key,
    mpp=mpp,
    labels_key=labels_key
)

# Quick confirmation
print(f"✅ Inserted StarDist labels under adata.obs['{labels_key}']")

## 6. Bin-to-Cell Conversion
Use `bin2cell` to aggregate bin-level transcript counts into single-cell resolution based on segmentation labels.

In [ ]:
# run bin_to_cell
cdata = b2c.bin_to_cell(
    adata,
    labels_key=labels_key,
    spatial_keys=basis_key
)

In [ ]:
cdata

In [ ]:
# the labels_he does not transfer over during bin_to_cell, so we need to merge them back in
# Create a mapping from adata.obs to cdata.obs using an integer ID

# Step 1: Add an integer index to adata.obs
adata.obs['int_index'] = np.arange(adata.n_obs)

# Step 2: Create the label mapping from adata (keeping 'labels_he')
label_map = adata.obs[['int_index', 'labels_he']].copy()

# Step 3: Merge label_map into cdata.obs using the integer ID
# Ensure object_id in cdata is int so the merge works
cdata.obs['object_id'] = cdata.obs['object_id'].astype(int)
label_map['int_index'] = label_map['int_index'].astype(int)

# Step 4: Perform the merge
cdata.obs = cdata.obs.merge(label_map, left_on='object_id', right_on='int_index', how='left')

# Step 5: Clean up (optional)
cdata.obs = cdata.obs.drop(columns=['int_index'])  # keep 'object_id' and 'labels_he'

In [ ]:
print(cdata)
print(cdata.obs.columns)

In [ ]:
# plot distribution of cell sizes
sns.histplot(cdata.obs['bin_count'], bins=100)
plt.yscale('log')
plt.xlabel("Number of Bins per Cell")
plt.ylabel("Cell Count (log scale)")
plt.title("Cell Size Distribution (log scale)")
plt.show()

In [ ]:
# filter cells by size
cdata = cdata[cdata.obs['bin_count'] >= 3].copy()

In [ ]:
cdata

In [ ]:
sc.pl.spatial(
    cdata,
    color=None,
    img_key=image_key,  # adjust if needed
    basis=key,
    title="Segmented Cells Overlay",
    size=8.0,
    alpha=0.8
)

## 7. Quality Control Metrics
Calculate standard QC metrics for cell-level filtering and downstream analysis.

In [ ]:
# recalc qc metrics
# revert cdata back to raw counts
sc.pp.calculate_qc_metrics(cdata, inplace=True)

In [ ]:
cdata.X = cdata.X.astype(np.int32)

## 8. Normalize and Identify HVGs
Normalize expression data and identify highly variable genes (HVGs).

In [ ]:
# select HVGs
sc.pp.highly_variable_genes(cdata, n_top_genes=top_variable_genes, flavor="seurat_v3")

In [ ]:
# normalize & log-transform
sc.pp.normalize_total(cdata, target_sum=1e4)
sc.pp.log1p(cdata)

In [ ]:
cdata

In [ ]:
cdata.obs['labels_he'] = cdata.obs['labels_he'].astype(str)

In [ ]:
# plot stardist
sc.pl.spatial(
    cdata,
    img_key=image_key,
    basis=key,
    color="labels_he",
    size=10,
    cmap="magma",
    legend_loc=None,
    title="StarDist Cell Labels"
)

In [ ]:
# only labeled cells plot
labeled = cdata[cdata.obs['labels_he'] != '0']

In [ ]:
# plot labeled cells
sc.pl.spatial(
    labeled,
    img_key=image_key,
    basis=key,
    color="labels_he",
    size=10,
    cmap="plasma",
    legend_loc=None,
    title="Labeled Stardist Cells"
)

In [ ]:
total_cells = cdata.n_obs
labeled_cells = np.sum(cdata.obs['labels_he'] != '0')
print(f"Labeled cells: {labeled_cells}/{total_cells} ({(labeled_cells/total_cells)*100:.2f}%)")

In [ ]:
# 1. Tag cells as 'Labeled' or 'Unlabeled'
cdata.obs['label_status'] = cdata.obs['labels_he'].astype(str).replace('0', 'Unlabeled')
cdata.obs['label_status'] = cdata.obs['label_status'].apply(lambda x: 'Unlabeled' if x == 'Unlabeled' else 'Labeled')

# 2. Create figure and axes
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

# 3. Labeled cells
sc.pl.spatial(
    cdata[cdata.obs['label_status'] == 'Labeled'],
    color="bin_count",
    img_key=image_key,
    basis="spatial_cropped_150_buffer",
    size=5,
    cmap="Greens",
    title="Labeled Cells",
    show=False,
    ax=axes[0],
    legend_loc=None
)

# 4. Unlabeled cells
sc.pl.spatial(
    cdata[cdata.obs['label_status'] == 'Unlabeled'],
    color="bin_count",
    img_key=image_key,
    basis="spatial_cropped_150_buffer",
    size=5,
    cmap="Reds",
    title="Unlabeled Cells",
    show=False,
    ax=axes[1],
    legend_loc=None
)

plt.tight_layout()
plt.show()

In [ ]:
# Filter out cells with label "0" (assuming it's the unassigned label)
cdata_labeled = cdata[cdata.obs['labels_he'] != '0'].copy()

## 9. PCA and Clustering
Perform dimensionality reduction and neighborhood graph construction for visualization and clustering.

In [ ]:
# run pca
sc.tl.pca(cdata_labeled, svd_solver='arpack')
# inspect
sc.pl.pca_variance_ratio(cdata_labeled, log=True, n_pcs=50)

In [ ]:
# compute neighborhood graph
sc.pp.neighbors(cdata_labeled, n_neighbors=10, n_pcs=20)

## 9. UMAP Embedding
Visualize cells using UMAP colored by segmentation labels or clusters.

In [ ]:
# perform UMAP
sc.tl.umap(cdata_labeled)
sc.pl.umap(cdata_labeled, color="labels_he", title="UMAP by StarDist Label", legend_loc=None)

## 10. Leiden Clustering
Visualize cells using UMAP colored by segmentation labels or clusters.

In [ ]:
# perform leiden clustering
sc.tl.leiden(cdata_labeled, resolution=0.5)
sc.pl.umap(cdata_labeled, color='leiden', title="Leiden Clusters")

In [ ]:
# spatial plot of leiden clusters
# Now plot spatial layout colored by cluster
sc.pl.spatial(
    cdata_labeled,
    color="leiden",                  # key from .obs
    img_key=image_key,              # previously determined image key
    basis=key,                      # previously determined spatial basis key
    cmap="tab20",                   # categorical colormap
    size=10,
    title="Leiden Cluster Overlay on Tissue",
    legend_loc="on data",           # or "right margin" if overlapping
)

In [ ]:
# define function for final prep before exporting anndata
def prepare_for_export(adata, spatial_key="spatial"):
    import numpy as np
    import pandas as pd

    # Convert categorical obs columns to strings
    for col in adata.obs.columns:
        if pd.api.types.is_categorical_dtype(adata.obs[col]):
            adata.obs[col] = adata.obs[col].astype(str)
    
    # Optional: keep only relevant keys
    # adata.uns = {k: adata.uns[k] for k in ['spatial']}  # optional pruning

    # Make sure spatial key is named "spatial" for R compatibility
    if spatial_key in adata.obsm and spatial_key != "spatial":
        adata.obsm["spatial"] = adata.obsm[spatial_key]

    return adata

In [ ]:
# prepare the AnnData object for export
cdata_prepped = prepare_for_export(cdata_labeled, spatial_key=key)

## 11. Save Processed Data
Write processed AnnData objects to disk for downstream analysis in Python or R.

In [ ]:
# Save the AnnData object with a descriptive filename
output_filename = f"{sample_id}_binned_qc_processed.h5ad"
cdata_prepped.write(output_filename, compression="gzip")